# 🚀 TrustChain Advanced Scenarios

This notebook covers **advanced integration patterns**:

1. LangChain Integration
2. Key Persistence & Rotation
3. Redis-backed Nonce Storage
4. Batch Operations & Streaming
5. Custom Verification Pipelines
6. Distributed Multi-Agent Setup

In [ ]:
import json
import os
import tempfile
import time

from trustchain import TrustChain, TrustChainConfig, TrustChainVerifier

## 1️⃣ LangChain Integration

Convert TrustChain tools to LangChain-compatible format.

In [ ]:
tc = TrustChain(TrustChainConfig(enable_nonce=False))

@tc.tool("database_query")
def query_database(query: str, limit: int = 10):
    """Execute database query with signed results."""
    return {
        "query": query,
        "rows": [{"id": i, "value": f"row_{i}"} for i in range(limit)],
        "count": limit
    }

@tc.tool("send_notification")
def send_notification(recipient: str, message: str, priority: str = "normal"):
    """Send notification with audit trail."""
    return {
        "recipient": recipient,
        "message": message,
        "priority": priority,
        "sent_at": time.time(),
        "status": "delivered"
    }

# Convert to LangChain format
try:
    from trustchain.integrations.langchain import to_langchain_tools

    lc_tools = to_langchain_tools(tc)
    print("🔗 LangChain tools created:")
    for tool in lc_tools:
        print(f"   • {tool.name}: {tool.description[:50]}...")
except ImportError:
    print("⚠️ LangChain not installed. Run: pip install langchain-core")

# Get OpenAI-compatible schema for function calling
schemas = tc.get_tools_schema()
print(f"\n📋 OpenAI function schemas: {len(schemas)} tools")
print(json.dumps(schemas[0], indent=2)[:300] + "...")

## 2️⃣ Key Persistence & Rotation

Save keys to disk and rotate them securely.

In [ ]:
# Create TrustChain with persistent keys
with tempfile.TemporaryDirectory() as tmpdir:
    key_file = os.path.join(tmpdir, "keys.json")

    # First instance - generates new keys
    tc1 = TrustChain(TrustChainConfig(
        key_file=key_file,
        enable_nonce=False
    ))
    tc1.save_keys()
    key_id_1 = tc1.get_key_id()

    @tc1.tool("test")
    def test_tool(x: int):
        return {"result": x * 2}

    signed1 = test_tool(5)
    print(f"🔑 Key ID: {key_id_1[:16]}...")
    print(f"📝 Signed response: {signed1.data}")

    # Second instance - loads existing keys
    tc2 = TrustChain(TrustChainConfig(
        key_file=key_file,
        enable_nonce=False
    ))
    key_id_2 = tc2.get_key_id()

    print(f"\n🔄 Second instance loaded same key: {key_id_1 == key_id_2}")
    print(f"✅ Can verify previous signature: {tc2.verify(signed1)}")

    # Key rotation - generates completely new keys
    print("\n🔄 Rotating keys...")
    new_key_id = tc2.rotate_keys()
    print(f"🆕 New Key ID: {new_key_id[:16]}...")
    print(f"❌ Old signature with new key: {tc2.verify(signed1)}")


## 3️⃣ Redis-backed Nonce Storage

Prevent replay attacks in distributed systems.

In [ ]:
# Redis config (requires running Redis server)
redis_url = os.getenv("REDIS_URL", "redis://localhost:6379")

try:
    tc_redis = TrustChain(TrustChainConfig(
        redis_url=redis_url,
        nonce_backend="redis",
        nonce_ttl=3600,  # 1 hour
        enable_nonce=True
    ))

    @tc_redis.tool("secure_action")
    def secure_action(action: str):
        return {"action": action, "executed": True}

    response = secure_action("transfer_funds")
    print(f"✅ Action executed with nonce: {response.nonce[:16]}...")
    print("🔐 Redis-backed replay protection active")

except Exception as e:
    print(f"⚠️ Redis not available: {e}")
    print("   Using memory-based nonce storage instead")

    tc_memory = TrustChain(TrustChainConfig(
        nonce_backend="memory",
        enable_nonce=True
    ))

    @tc_memory.tool("secure_action")
    def secure_action_mem(action: str):
        return {"action": action, "executed": True}

    response = secure_action_mem("transfer_funds")
    print(f"✅ Action with memory nonce: {response.nonce[:16]}...")

## 4️⃣ Batch Operations & Streaming

Sign multiple operations efficiently.

In [ ]:
tc_batch = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_batch.tool("process_item")
def process_item(item_id: int, data: str):
    return {"id": item_id, "processed": data.upper(), "ts": time.time()}

# Batch processing
items = [(i, f"data_{i}") for i in range(20)]

start = time.perf_counter()
results = [process_item(id, data) for id, data in items]
elapsed = (time.perf_counter() - start) * 1000

print(f"📦 Batch processed {len(results)} items in {elapsed:.2f}ms")
print(f"⚡ {elapsed/len(results):.2f}ms per item")
print(f"✅ All verified: {all(r.is_verified for r in results)}")

# Streaming with generator
def stream_processor(count: int):
    for i in range(count):
        yield process_item(i, f"stream_{i}")

print("\n🌊 Streaming mode:")
for i, result in enumerate(stream_processor(5)):
    print(f"   Item {i}: sig={result.signature[:16]}... ✓")

## 5️⃣ Custom Verification Pipeline

Build complex verification workflows.

In [ ]:
tc_verify = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_verify.tool("financial_transaction")
def create_transaction(amount: float, currency: str, recipient: str):
    return {
        "amount": amount,
        "currency": currency,
        "recipient": recipient,
        "tx_id": f"TX-{int(time.time())}"
    }

# Create a transaction
tx = create_transaction(1000.0, "USD", "vendor@example.com")

# Create standalone verifier (can be on different machine)
verifier = TrustChainVerifier(tc_verify.export_public_key())

print("🔍 Verification Pipeline:")
print(f"   1. Transaction created: {tx.data['tx_id']}")

# Step 1: Signature verification
is_valid = verifier.verify(tx)
print(f"   2. Signature valid: {is_valid}")

# Step 2: Business rules check
amount_ok = tx.data["amount"] <= 10000
print(f"   3. Amount within limit: {amount_ok}")

# Step 3: Timestamp freshness
is_fresh = (time.time() - tx.timestamp) < 300  # 5 min
print(f"   4. Timestamp fresh: {is_fresh}")

# Final verdict
approved = is_valid and amount_ok and is_fresh
print(f"\n{'✅' if approved else '❌'} Transaction {'APPROVED' if approved else 'REJECTED'}")

## 6️⃣ Distributed Multi-Agent Setup

Multiple AI agents with independent key pairs.

In [ ]:
from trustchain.v2.tenants import TenantManager

# Central agent registry
manager = TenantManager()

# Create specialized agents
agents = {
    "research_agent": manager.get_or_create("research_agent"),
    "code_agent": manager.get_or_create("code_agent"),
    "review_agent": manager.get_or_create("review_agent"),
}

print("🤖 Distributed Agent Setup:")
for name, tc in agents.items():
    print(f"   {name}: key={tc.get_key_id()[:12]}...")

# Simulate multi-agent workflow
print("\n📋 Multi-Agent Workflow:")

# Research agent gathers info
research_result = agents["research_agent"]._signer.sign(
    "research", {"topic": "quantum computing", "sources": 15}
)
print(f"   1. Research: {research_result.data['sources']} sources found")

# Code agent writes implementation
code_result = agents["code_agent"]._signer.sign(
    "implementation",
    {"language": "python", "lines": 150, "parent": research_result.signature[:16]}
)
print(f"   2. Code: {code_result.data['lines']} lines written")

# Review agent validates
review_result = agents["review_agent"]._signer.sign(
    "review",
    {
        "approved": True,
        "research_valid": agents["research_agent"].verify(research_result),
        "code_valid": agents["code_agent"].verify(code_result)
    }
)
print(f"   3. Review: approved={review_result.data['approved']}")

# Cross-agent verification
print("\n🔐 Cross-Agent Verification:")
can_research_verify_code = agents["research_agent"].verify(code_result)
can_code_verify_own = agents["code_agent"].verify(code_result)
print(f"   Research→Code: {can_research_verify_code} (expected: False)")
print(f"   Code→Code: {can_code_verify_own} (expected: True)")

## 📚 Next Steps

- See `trustchain_pro.ipynb` for enterprise features
- Check [API Reference](https://github.com/petro1eum/trust_chain/wiki/API-Reference)
- Deploy with [MCP Server](https://github.com/petro1eum/trust_chain/wiki/Examples#mcp-server)